In [2]:
from preproc_functions_controls import *

In [3]:
root_path = '/Users/dg519/Documents/normative_paper_github/data'
remote_data_folders='/data_ic3online_cognition'
supervised_data_folders=['/data_healthy_v1','/data_healthy_v2']
folder_structure=['/summary_data','/trial_data','/speech']
output_clean_folder ='/data_healthy_cleaned'
list_of_tasks = ['IC3_Orientation', 'IC3_PearCancellation', 'IC3_rs_digitSpan', 'IC3_rs_spatialSpan', 'IC3_rs_PAL', 'IC3_rs_SRT', 'IC3_rs_CRT', 'IC3_SemanticJudgment', 'IC3_i4i_IDED', 'IC3_i4i_motorControl','IC3_calculation', 'IC3_GestureRecognition', 'IC3_AuditorySustainedAttention','IC3_BBCrs_blocks', 'IC3_Comprehension', 'IC3_NVtrailMaking','IC3_rs_oddOneOut', 'IC3_TaskRecall']
list_of_questionnaires = ['q_IC3_demographics', 'q_IC3_fatigue','q_IC3_GDS','q_IC3_apathy']
list_of_speech = ['IC3_NamingTest','IC3_Reading', 'IC3_Repetition']
folder_list = ['data_healthy_v1', 'data_healthy_v2', 'data_ic3online_cognition', 'data_healthy_merged']

In [ ]:
main_preprocessing(root_path, list_of_tasks, list_of_questionnaires, list_of_speech)

In [1]:
from preproc_functions_patients import *

In [2]:
root_path = '/Users/dg519/Documents/normative_paper_github/data'
list_of_tasks = ['IC3_Orientation', 'IC3_PearCancellation', 'IC3_rs_digitSpan', 'IC3_rs_spatialSpan', 'IC3_rs_PAL', 'IC3_rs_SRT', 'IC3_rs_CRT', 'IC3_SemanticJudgment', 'IC3_i4i_IDED', 'IC3_i4i_motorControl','IC3_calculation', 'IC3_GestureRecognition', 'IC3_AuditorySustainedAttention','IC3_BBCrs_blocks', 'IC3_Comprehension', 'IC3_NVtrailMaking','IC3_rs_oddOneOut', 'IC3_TaskRecall']
list_of_questionnaires = ['q_IC3_demographics', 'q_IC3_fatigue','q_IC3_GDS','q_IC3_apathy', 'q_IC3_IADL']
list_of_speech = ['IC3_NamingTest','IC3_Reading', 'IC3_Repetition']
patient_data_folders=['/data_patients_v1','/data_patients_v2']
folder_structure=['/summary_data','/trial_data','/speech']
output_clean_folder ='/data_patients_cleaned'
merged_data_folder ='/data_patients_merged'
clean_file_extension='_cleaned'
data_format='.csv'
clinical_information = '/Users/dg519/Documents/normative_paper_github/data/output_data/patients_database.xlsx'


In [3]:
df= main_preprocessing(root_path, list_of_tasks, list_of_questionnaires, list_of_speech, clinical_information)

Starting preprocessing...
Merging data across sites...Merged IC3_Orientation
Merged IC3_PearCancellation
Merged IC3_rs_digitSpan
Merged IC3_rs_spatialSpan
Merged IC3_rs_PAL
Merged IC3_rs_SRT
Merged IC3_rs_CRT
Merged IC3_SemanticJudgment
Merged IC3_i4i_IDED
Merged IC3_i4i_motorControl
Merged IC3_calculation
Merged IC3_GestureRecognition
Merged IC3_AuditorySustainedAttention
Merged IC3_BBCrs_blocks
Merged IC3_Comprehension
Merged IC3_NVtrailMaking
Merged IC3_rs_oddOneOut
Merged IC3_TaskRecall
Merged IC3_NamingTest
Merged IC3_Reading
Merged IC3_Repetition
Merged q_IC3_demographics
Merged q_IC3_fatigue
Merged q_IC3_GDS
Merged q_IC3_apathy
Merged q_IC3_IADL
Done
Pre-processing IC3_Orientation...Done
Pre-processing IC3_PearCancellation...Done
Pre-processing IC3_rs_digitSpan...Done
Pre-processing IC3_rs_spatialSpan...Done
Pre-processing IC3_rs_PAL...Done
Pre-processing IC3_rs_SRT...Done
Pre-processing IC3_rs_CRT...Done
Pre-processing IC3_SemanticJudgment...Done
Pre-processing IC3_i4i_IDED...D